In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import os

def import_from_excel(e_file, filename):
    kinase_targets = e_file['kinase target'].tolist()
    if kinase_targets != []:
        kinase_targets = kinase_targets[0]#just the top score
    JS_scores = e_file['JS Distance Score'].tolist() #just the top score
    if JS_scores != []:
        JS_scores = JS_scores[0]
    #off_targets = e_file['off-target %inhibition values'].tolist()
    mu = int(filename.split('.')[0].split('_')[-2])
    return [mu, kinase_targets, JS_scores] #off_targets]

test_results_200 = []
test_results_700 = []

test_results = {}

for filename in os.listdir(os.getcwd()+'/klaeger_kinasepair_scan_results/'):
    if filename[0:8] == '030822_i':
        f = './'+str('./klaeger_kinasepair_scan_results/'+filename)
        replicate_results = []
        targets = ''
        for x in range(1,4):
            open_file = pd.read_excel(f, str(x), engine='openpyxl')
            inhibitor_combination = import_from_excel(open_file, filename)
            targets = inhibitor_combination[1]
            replicate_results.append((x,inhibitor_combination))
        if targets != []:
            if targets not in test_results:
                test_results[targets] = [[],[]]
            if replicate_results[0][1][0] == 200:
                test_results[targets][0].append(replicate_results)
                #test_results_200.append(replicate_results)
            elif replicate_results[0][1][0] == 700:
                test_results[targets][1].append(replicate_results)
                #test_results_700.append(replicate_results)
        
#check to make sure we have 5 sets of results for each  
for k in test_results:
    print(len(test_results[k][0]))
    print(len(test_results[k][1]))

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
3
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
3
0


In [7]:
#sig results

sig_results = {}

for k in test_results:
    res_200 = test_results[k][0]
    res_700 = test_results[k][1]
    
    #get the best i=1 or i=2 score to compare to... either the most selective single inhibitor
    #or the most selective single inhibitor for each kinase
    
    #get the average i=1 score (most selective )
    i_1_200_vals = [r[0][1][2] for r in res_200 if r[0][0] == 1]
    i_1_700_vals = [r[0][1][2] for r in res_700 if r[0][0] == 1]
    
    i_2_200_vals = [r[1][1][2] for r in res_200 if r[1][0] == 2]
    i_2_700_vals = [r[1][1][2] for r in res_700 if r[1][0] == 2]
        
    sig_200_scores = []
    sig_700_scores = []
    
    sig_dist_200 = 0
    sig_dist_700 = 0
    
    if res_200 != []:
    
        #get the vals at each higher combo number. test if sig. if so, save those vals
        for i in range(1, len(res_200[0])):
            
            i_vals = [r[i][1][2] for r in res_200]
            
            #compare to i=1 or i=2, must be significantly better than both
            test_set = []
            if np.mean(i_1_200_vals) > np.mean(i_2_200_vals):
                test_set = i_1_200_vals
            else:
                test_set = i_2_200_vals
                
            #do we have a test set? if not, automatically significant
            if test_set != []:
            
                #test if sig diff or not
                tval, pval = stats.ttest_ind(test_set, i_vals)
            
                #if sig and absolute diff is > 0.001
                #if (pval < 0.05) and (np.mean(test_set)+0.001 < np.mean(i_vals)):
                if np.mean(test_set)+0.001 < np.mean(i_vals):
                    
                    #if this is better than our previous best score...
                    if np.mean(i_vals) - np.mean(test_set) > sig_dist_200:
                        sig_200_scores = i_vals
                        sig_dist_200 = (np.mean(i_vals) - np.mean(test_set))   
            else:
                #so we don't have a test set, will be significant, check to see if it is beter than our last
                    if np.mean(i_vals) > sig_dist_200:
                        sig_200_scores = i_vals
                        sig_dist_200 = np.mean(i_vals)
                        
    if res_700 != []:
    
        #get the vals at each higher combo number. test if sig. if so, save those vals
        for i in range(1, len(res_700[0])):
            
            i_vals = [r[i][1][2] for r in res_700]
            
            #compare to i=1 or i=2, must be significantly better than both
            test_set = []
            if np.mean(i_1_700_vals) > np.mean(i_2_700_vals):
                test_set = i_1_700_vals
            else:
                test_set = i_2_700_vals
                
            #do we have a test set? if not, automatically significant
            if test_set != []:
            
                #test if sig diff or not
                tval, pval = stats.ttest_ind(test_set, i_vals)
            
                #if sig and absolute diff is > 0.001
                if (pval < 0.05) and (np.mean(test_set)+0.001 < np.mean(i_vals)):
                    
                    #if this is better than our previous best score...
                    if np.mean(i_vals) - np.mean(test_set) > sig_dist_700:
                        sig_700_scores = i_vals
                        sig_dist_700 = np.mean(i_vals) - np.mean(test_set)  
            else:
                #so we don't have a test set, will be significant, check to see if it is beter than our last
                    if np.mean(i_vals) > sig_dist_700:
                        sig_700_scores = i_vals
                        sig_dist_700 = np.mean(i_vals)  
                        
    if sig_200_scores != [] or sig_700_scores != []:
        knames = ''
        if test_results[k][1] != []:
            for iter in test_results[k][1][0]:
                if iter[1][1] != []:
                    knames = iter[1][1]
        else:
            for iter in test_results[k][0][0]:
                if iter[1][1] != []:
                    knames = iter[1][1]
        knames = knames.strip('][').replace('\'', '').replace(', ', '_')
        sig_results[knames] = [sig_200_scores,sig_700_scores,sig_dist_200,sig_dist_700]
        
for key in sig_results:
    print(key, sig_results[key])
   
print( 'Kinase-Pair', 'JSD-Score-tight-prior', 'JSD-score-medium-prior')
for key in sig_results:
    if sig_results[key][2]>0 and sig_results[key][3]>0:
        print(key, sig_results[key][2], sig_results[key][3])

TNIK_STK3 [[0.8595269418945408, 0.8604229012633942, 0.8579741916610377], [0.7767828652611855, 0.7780019478231244, 0.7764300130055635], 0.003019701016570564, 0.004373285615347444]
ADCK1_TANK [[], [0.737479403465644, 0.7410089059820573, 0.7393763932900169], 0, 0.005902281463063042]
CDK1_CDK7 [[0.8755743537700883, 0.8763203272060233, 0.8735136192010298], [0.8201354806479775, 0.8200762102719743, 0.8189688891907385], 0.009331763527266057, 0.005841983172624521]
EPHA4_PLK4 [[0.8870591881748773, 0.8905849144250889, 0.8871814210961264], [0.8154516315975694, 0.8133992600823482, 0.8123406131356276], 0.004410891614712087, 0.005742405296916386]
HCK_CSK [[], [0.7880975198073473, 0.7875872062541318, 0.7861938297017592], 0, 0.0037316874492043794]
MAP3K11_ULK1 [[0.8519874676769537, 0.8547266472376719, 0.8522488391623051], [0.7763655387995244, 0.7777175748996767, 0.7753763672297235], 0.0073807403315504505, 0.005846740395307082]
PAK4_MARK3 [[0.8749611444705733, 0.8726246000276924, 0.8753544229144283], [0